In [394]:
with open('18a.txt') as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]

In [395]:
first = lines[0]
# TODO: implement the split mechanism and then full reducing mechanism. then make addition and magnitude.

In [396]:
import math
from typing import Tuple, List

In [397]:
def string_exploder(s: str, left_idx: int) -> str:
    ldd = False
    rdd = False
    near_left_dd = False
    near_right_dd = False
    if s[left_idx+1].isnumeric():
        ldd = True
    if ldd:
        left_val = s[left_idx] + s[left_idx+1]
    else:
        left_val = s[left_idx]
    right_val = s[left_idx+2+ldd]
    if s[left_idx+3+ldd].isnumeric():
        rdd = True
    if rdd:
        right_val = s[left_idx+2+ldd] + s[left_idx+3+ldd]
    left_double_digits = False
    for i in range(left_idx-1, 0, -1):
        if s[i].isnumeric():
            near_left_val = int(s[i])
            if s[i-1].isnumeric():
                near_left_dd = True
                near_left_val = int(s[i-1]+s[i])
                s = s[:i-1] + str(near_left_val + int(left_val)) + s[i+1:]
                break
            elif int(s[i]) + int(left_val) >= 10: left_double_digits = True
            s = s[:i] + str(int(s[i]) + int(left_val)) + s[i+1:]
            break
    for j in range(left_idx+4+ldd+rdd, len(s)-1):
        if s[j].isnumeric():
            near_right_val = int(s[j])
            # print(near_right_val)
            if s[j+1].isnumeric():
                near_right_val = int(s[j] + s[j+1])
                s = s[:j] + str(near_right_val + int(right_val)) + s[j+2:]
            # print("sj: " + str(s[j]))
            # print("right_val: " + right_val)
            else:
                s = s[:j] + str(int(s[j]) + int(right_val)) + s[j+1:]
            # print(int(s[j]) + int(right_val))
            # print("oh yeah")
            break
    s = s[:left_idx-1+left_double_digits] + '0' + s[left_idx+4+left_double_digits+ldd+rdd:]
    return s

In [398]:
def string_nest_search(s: str) -> Tuple[str, bool]:
    # what if i do this all in string form?
    bracket_counter = 0
    for i in range(len(s)):
        c = s[i]
        if c == '[': bracket_counter += 1
        elif c == ']': bracket_counter -= 1
        elif c.isnumeric():
            if bracket_counter > 4 and i + 3 < len(s) and s[i+1].isnumeric() and s[i+2] == ',' and s[i+3].isnumeric():
                s = string_exploder(s, i)
                return (s, True)
            if bracket_counter > 4 and i + 2 < len(s) and s[i+1] == ',' and s[i+2].isnumeric():
                # print(c)
                # print("INDEX: " + str(i))
                s = string_exploder(s, i)
                return (s, True)
            
    return (s, False)

In [399]:
def string_splitter(s: str, left_idx: int) -> str:
    num = int(s[left_idx:left_idx+2])
    left_val = math.floor(num/2.0)
    right_val = math.ceil(num/2.0)
    s = s[:left_idx] + '[' + str(left_val) + ',' + str(right_val) + ']' + s[left_idx+2:]
    return s


In [400]:
def string_dd_search(s: str) -> Tuple[str, bool]:
    # search for numbers greater than or equal to 10
    for i in range(len(s)-1):
        if s[i].isnumeric() and s[i+1].isnumeric():
            s = string_splitter(s,i)
            return (s, True)
    return (s, False)

In [401]:
def add(s1: str, s2: str) -> str:
    return '[' + s1 + ',' + s2 + ']'

In [402]:
def reducer(s: str) -> str:
    done = False
    while not done:
        # print(s)
        s, change = string_nest_search(s)
        if not change:
            s, change = string_dd_search(s)
        if not change: done = True
    return s

In [403]:
def add_and_reduce(nums: List[str]) -> str:
    final_num = nums[0]
    for i in range(1, len(nums)):
        # print(final_num)
        final_num = add(final_num, nums[i])
        # print(final_num)
        final_num = reducer(final_num)
        # print(final_num)
    return final_num

In [404]:
class Pair:

    def __init__(self, left_val, right_val, depth, final_idx):
        self.left_val = left_val
        self.right_val = right_val
        self.depth = depth
        self.final_idx = final_idx
        self.literal = False
        if isinstance(self.left_val, int) and isinstance(self.right_val, int): self.literal = True
    
    def __str__(self):
        return "[" + str(self.left_val) + "," + str(self.right_val) + "]"
    
    def get_magnitude(self):
        if isinstance(self.left_val,int): l_raw_mag = self.left_val
        else: l_raw_mag = self.left_val.get_magnitude()
        if isinstance(self.right_val, int): r_raw_mag = self.right_val
        else: r_raw_mag = self.right_val.get_magnitude()
        return 3 * l_raw_mag + 2 * r_raw_mag

In [405]:
def parse_string(s: str, d: int):
    # return the pair it represents and the index where it ends.
    if s[0] != '[': print('ps[0]: ' + s[0])
    # print(s[1])
    if s[1].isnumeric():
        # print("left val: " + s[1])
        left_val = int(s[1])
        right_idx = 3
    else:
        left_val, right_idx = parse_string(s[1:], d+1)
        right_idx += 3
    if s[right_idx].isnumeric():
        # print("right val: " + s[right_idx])
        right_val = int(s[right_idx])
        final_idx = right_idx
    else:
        right_val, final_idx = parse_string(s[right_idx:], d+1)
        final_idx += right_idx
    pair = Pair(left_val, right_val, d, final_idx)
    end_idx = final_idx + 1
    return pair, end_idx
    

In [406]:
def magnitude(s: str) -> int:
    outer_pair,_ = parse_string(s,0)
    return outer_pair.get_magnitude()

### Part 1

In [410]:
final_str = add_and_reduce(lines)
print(final_str)
print(magnitude(final_str))

[[[[6,6],[7,7]],[[7,7],[7,7]]],[[[9,7],[6,8]],[[7,7],[0,8]]]]
4184


### Part 2

In [413]:
duos = []
for i in range(len(lines)):
    for j in range(len(lines)):
        if i != j: duos.append([lines[i], lines[j]])

In [414]:
max_mag = 0
for duo in duos:
    reduced_str = add_and_reduce(duo)
    mag = magnitude(reduced_str)
    if mag > max_mag: max_mag = mag
print(max_mag)

4731
